In [1]:
!wget https://huggingface.co/lllyasviel/Annotators/resolve/main/RealESRGAN_x4plus.pth

--2025-01-20 06:56:45--  https://huggingface.co/lllyasviel/Annotators/resolve/main/RealESRGAN_x4plus.pth
Resolving huggingface.co (huggingface.co)... 18.239.50.80, 18.239.50.16, 18.239.50.103, ...
Connecting to huggingface.co (huggingface.co)|18.239.50.80|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/a2/11/a211ef997fa284ad67caa591ca931692f9a8a018910b18a04d9e6dff396f02e3/4fa0d38905f75ac06eb49a7951b426670021be3018265fd191d2125df9d682f1?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27RealESRGAN_x4plus.pth%3B+filename%3D%22RealESRGAN_x4plus.pth%22%3B&Expires=1737615405&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNzYxNTQwNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9hMi8xMS9hMjExZWY5OTdmYTI4NGFkNjdjYWE1OTFjYTkzMTY5MmY5YThhMDE4OTEwYjE4YTA0ZDllNmRmZjM5NmYwMmUzLzRmYTBkMzg5MDVmNzVhYzA2ZWI0OWE3OTUxYjQyNjY3MDAyMWJlMzAxODI2NWZkMTkxZDIxMjVkZjlkNjgyZjE%7EcmVzcG9uc2Ut

In [3]:
pip install pytesseract realesrgan --quiet

In [9]:
 pip install torchvision==0.16.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [1]:
import cv2
import torch
import numpy as np
import pytesseract
from realesrgan import RealESRGAN

def enhance_plate_for_ocr(roi: np.ndarray, model_path='RealESRGAN_x4plus.pth') -> np.ndarray:
    """
    Enhance a BGR ROI array using Real-ESRGAN and return enhanced BGR image.
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    # Convert to RGB
    roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)

    model = RealESRGAN(device, scale=4)
    model.load_weights(model_path)

    # Run inference
    with torch.no_grad():
        enhanced_rgb = model.predict(roi_rgb)

    # Convert back to BGR
    enhanced_bgr = cv2.cvtColor(enhanced_rgb, cv2.COLOR_RGB2BGR)
    return enhanced_bgr

def preprocess_for_ocr(img_bgr: np.ndarray) -> np.ndarray:
    """
    Optional additional preprocessing (grayscale, threshold, etc.) for OCR.
    """
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    # Maybe do some denoising if needed
    # gray = cv2.medianBlur(gray, 3)

    # Adaptive threshold or Otsu
    # For demonstration:
    thresh = cv2.adaptiveThreshold(gray, 255,
                                   cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 11, 2)

    return thresh

def read_plate_text(roi_bgr: np.ndarray, model_path='RealESRGAN_x4plus.pth') -> str:
    # 1. Enhance using ESRGAN
    enhanced_roi = enhance_plate_for_ocr(roi_bgr, model_path=model_path)

    # 2. Additional classical preprocessing
    prep_roi = preprocess_for_ocr(enhanced_roi)

    # 3. OCR
    config = "--psm 7" # Single text line assumption, tweak as needed
    text = pytesseract.image_to_string(prep_roi, config=config)

    return text.strip()

if __name__ == "__main__":
    # Example usage: we already have a cropped ROI named "roi_plate.jpg"
    roi_bgr = cv2.imread("roi_plate.jpg")
    recognized_text = read_plate_text(roi_bgr, model_path="RealESRGAN_x4plus.pth")
    print("OCR result:", recognized_text)


/usr/local/lib/python3.11/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(


ImportError: cannot import name 'RealESRGAN' from 'realesrgan' (/usr/local/lib/python3.11/dist-packages/realesrgan/__init__.py)

In [7]:
!pip install basicsr --quiet  # Ensure basicsr is installed

import basicsr.data.degradations
import os

# Get the path to the degradations.py file
file_path = basicsr.data.degradations.__file__

# Read the content of the file
with open(file_path, 'r') as f:
    file_content = f.read()

# Replace the old import with the new one
new_content = file_content.replace(
    "from torchvision.transforms.functional_tensor import rgb_to_grayscale",
    "from torchvision.transforms.functional import rgb_to_grayscale"
)

# Write the modified content back to the file
with open(file_path, 'w') as f:
    f.write(new_content)

print(f"Modified {file_path}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 820.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 112.4 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'

In [1]:
!git clone https://github.com/xinntao/Real-ESRGAN.git
!cd Real-ESRGAN

Cloning into 'Real-ESRGAN'...
remote: Enumerating objects: 759, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 759 (delta 0), reused 0 (delta 0), pack-reused 757 (from 2)
Receiving objects: 100% (759/759), 5.39 MiB | 16.71 MiB/s, done.
Resolving deltas: 100% (406/406), done.


In [3]:
!python /content/Real-ESRGAN/inference_realesrgan.py -n RealESRGAN_x4plus -i '/content/C.jpeg' --face_enhance

Traceback (most recent call last):
  File "/content/Real-ESRGAN/inference_realesrgan.py", line 5, in <module>
    from basicsr.archs.rrdbnet_arch import RRDBNet
  File "/usr/local/lib/python3.11/dist-packages/basicsr/__init__.py", line 4, in <module>
    from .data import *
  File "/usr/local/lib/python3.11/dist-packages/basicsr/data/__init__.py", line 22, in <module>
    _dataset_modules = [importlib.import_module(f'basicsr.data.{file_name}') for file_name in dataset_filenames]
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/basicsr/data/__init__.py", line 22, in <listcomp>
    _dataset_modules = [importlib.import_module(f'basicsr.data.{file_name}') for file_name in dataset_filenames]
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(na

In [6]:
!cd Real-ESRGAN

In [2]:
pip install basicsr>=1.4.2